In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
from hyperopt import fmin, tpe, hp

2023-10-29 10:46:08.900514: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-29 10:46:08.954974: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-29 10:46:08.955019: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-29 10:46:08.955069: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-29 10:46:08.964810: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-29 10:46:08.965719: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [ ]:
%pip install hyperopt

In [ ]:
%pip install tensorflow

In [ ]:
%pip install keras

In [2]:
# Load Dataset into the Data Frame
df = pd.read_csv('../data/data.csv')

In [3]:
df

,Date,GemstoneName,Color,Clarity,Cut,Weight,Price
0,2021-01-15,Spinel,Brown,Transparent,Oval,1.36,9000
1,2021-01-15,Cat's Eye,Yellow,Translucent,Oval,0.75,18000
2,2021-01-17,Spinel,Brown,Transparent,Oval,1.36,9000
3,2021-01-28,Cat's Eye,Yellow,Translucent,Oval,0.75,18000
4,2021-02-12,Sapphire,Purple,Transparent,Emerald,0.73,20000
...,...,...,...,...,...,...,...
696,2022-05-11,Ruby,Red,Transparent,Emerald,0.51,30000
697,2022-08-04,Ruby,Red,Transparent,Emerald,0.95,30000
698,2022-09-20,Ruby,Purple,Transparent,Oval,0.73,30000
699,2023-01-17,Ruby,Purple,Transparent,Oval,0.73,30000


In [4]:
# Encode categorical features
categorical_columns = ['GemstoneName','Color' , 'Clarity','Cut']
for column in categorical_columns:
    df[column] = LabelEncoder().fit_transform(df[column])

In [5]:
# Data Preparation
df['Date'] = pd.to_datetime(df['Date'])  # Convert 'Date' column to datetime
df.set_index('Date', inplace=True)  # Set 'Date' as the index

In [6]:
# Split the data into features and target
X = df.drop(columns=['Price'])
y = df['Price']

In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Define a hyperparameter search space
space = {
    'n_hidden_layers': hp.choice('n_hidden_layers', [1, 2, 3]),
    'n_units': hp.choice('n_units', [64, 128, 256]),
    'learning_rate': hp.loguniform('learning_rate', -5, -2),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5),
    'epochs': hp.quniform('epochs', 50, 200, 10),
    'batch_size': hp.choice('batch_size', [32, 64, 128])
}

In [9]:
# Define the objective function for hyperparameter tuning
def objective(params):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(X_train.shape[1],)))
    
    for _ in range(params['n_hidden_layers']):
        model.add(keras.layers.Dense(params['n_units'], activation='relu'))
        model.add(keras.layers.Dropout(params['dropout_rate']))
    
    model.add(keras.layers.Dense(1))  # Output layer for regression
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=params['learning_rate']),
                  loss='mean_squared_error')
    
    model.fit(X_train, y_train, epochs=int(params['epochs']), batch_size=params['batch_size'], verbose=0)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return rmse

In [10]:
# Perform hyperparameter tuning using Hyperopt
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100)

5/5 [==============================] - 0s 1ms/step    

5/5 [==============================] - 0s 1ms/step                             

5/5 [==============================] - 0s 1ms/step                             

5/5 [==============================] - 0s 2ms/step                             

5/5 [==============================] - 0s 1ms/step                             

5/5 [==============================] - 0s 2ms/step                          

5/5 [==============================] - 0s 2ms/step                              

5/5 [==============================] - 0s 2ms/step                              

5/5 [==============================] - 0s 2ms/step                              

5/5 [==============================] - 0s 2ms/step                              

5/5 [==============================] - 0s 2ms/step                               

5/5 [==============================] - 0s 2ms/step                               

5/5 [==============================] - 0s 2ms/st

In [11]:
# Train the final model with the best hyperparameters
final_model = keras.Sequential()
final_model.add(keras.layers.Input(shape=(X_train.shape[1],)))

In [12]:
for _ in range(best['n_hidden_layers']):
    final_model.add(keras.layers.Dense(best['n_units'], activation='relu'))
    final_model.add(keras.layers.Dropout(best['dropout_rate']))

final_model.add(keras.layers.Dense(1))

final_model.compile(optimizer=keras.optimizers.Adam(learning_rate=best['learning_rate']),
                    loss='mean_squared_error')

final_model.fit(X_train, y_train, epochs=int(best['epochs']), batch_size=best['batch_size'])

Epoch 1/180


18/18 [==============================] - 0s 1ms/step - loss: 9989784576.0000
Epoch 2/180
18/18 [==============================] - 0s 1ms/step - loss: 9987928064.0000
Epoch 3/180
18/18 [==============================] - 0s 1ms/step - loss: 9983314944.0000
Epoch 4/180
18/18 [==============================] - 0s 1ms/step - loss: 9974896640.0000
Epoch 5/180
18/18 [==============================] - 0s 1ms/step - loss: 9959266304.0000
Epoch 6/180
18/18 [==============================] - 0s 1ms/step - loss: 9934757888.0000
Epoch 7/180
18/18 [==============================] - 0s 1ms/step - loss: 9897792512.0000
Epoch 8/180
18/18 [==============================] - 0s 1ms/step - loss: 9841228800.0000
Epoch 9/180
18/18 [==============================] - 0s 1ms/step - loss: 9773317120.0000
Epoch 10/180
18/18 [==============================] - 0s 1ms/step - loss: 9681143808.0000
Epoch 11/180
18/18 [==============================] - 0s 1ms/step - loss: 9577986048.0000
Epoch 12/180
18/18 [===========

In [13]:
# Evaluate the final model
test_loss = final_model.evaluate(X_test, y_test)
print(f"Test RMSE: {np.sqrt(test_loss)}")

5/5 [==============================] - 0s 1ms/step - loss: 5001542656.0000
Test RMSE: 70721.58550258895


In [14]:
# Assuming you have trained and evaluated your final model as 'final_model'

# Save the model to a file
final_model.save('gemstone_price_prediction_model2.h5')


/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras

# Load the trained model
loaded_model = keras.models.load_model('gemstone_price_prediction_model2.h5')

# Collect user input
gemstone_name = input("Enter the gemstone name: ")
cut = input("Enter the cut: ")
color = input("Enter the color: ")
clarity = input("Enter the clarity: ")
weight = float(input("Enter the weight: "))

# Create a DataFrame with the user input
user_input = pd.DataFrame({
    'gemstone_name': [gemstone_name],
    'cut': [cut],
    'color': [color],
    'clarity': [clarity],
    'weight': [weight]
})

# Perform data preprocessing on the user input (similar to the training data)
categorical_columns = ['gemstone_name', 'cut', 'color', 'clarity']
for column in categorical_columns:
    user_input[column] = LabelEncoder().fit_transform(user_input[column])

numerical_columns = ['weight']
scaler = StandardScaler()
user_input[numerical_columns] = scaler.fit_transform(user_input[numerical_columns])

# Convert the input array to a float32 array
user_input = user_input.astype('float32')

# Make predictions using the loaded model
predicted_prices = loaded_model.predict(user_input.values)

# Display the predicted price
predicted_price = predicted_prices[0][0]
print(f'Predicted Gemstone Price: ${predicted_price:.2f}')


1/1 [==============================] - 0s 34ms/step
Predicted Gemstone Price: $34170.90
